In [34]:
# 1. Instalación de Librerías (Descomentar y ejecutar solo si es necesario)
# !pip install selenium pandas beautifulsoup4

# 2. Importación de Librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [35]:
# URL base de la liga chilena 2024
URL_LIGA_CHILENA_2024 = 'https://www.flashscore.com/football/chile/liga-de-primera-2024/results/'

def obtener_urls_partidos_flashscore(url_liga_general):
    """
    Usa Selenium para cargar la tabla de resultados dinámica y extraer los IDs/enlaces de partidos.
    """
    print("Iniciando Selenium para cargar la tabla dinámica...")
    
    # Asegúrate de que tu WebDriver (ej. ChromeDriver) esté configurado.
    driver = webdriver.Chrome() 
    driver.get(url_liga_general)
    driver.implicitly_wait(10)
    
# CÓDIGO CORREGIDO PARA HACER CLIC EN "SHOW MORE MATCHES"

    # 1. Hacer clic en "Show more matches" para cargar la temporada completa
    try:
        # Usamos WebDriverWait y By.XPATH para encontrar el elemento por su texto visible.
        mostrar_mas_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Show more matches')]"))
        )
        # Hacemos clic repetidamente hasta que el botón desaparezca (o falle)
        while True:
            try:
                # El botón tiene que ser clickeable para que la acción sea exitosa
                driver.execute_script("arguments[0].click();", mostrar_mas_button)
                time.sleep(2) # Pausa para que la nueva tabla cargue
            except Exception:
                # Si el clic falla (porque ya cargó todo y el botón desapareció), salimos.
                break
    except Exception:
        print("Botón 'Show more matches' no encontrado o ya cargó todo.")
        pass # Continuamos con el scraping de lo que se haya cargado.
    
    # ... (El resto del código para obtener el HTML sigue igual) ...
    
    # 2. Obtener el HTML final y cerrar el navegador
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit() 
    
    # 3. Extracción de Enlaces usando BeautifulSoup
    live_table = soup.find('div', id='live-table')
    if not live_table:
        print("Error: La tabla #live-table no se cargó correctamente.")
        return []

    filas_partidos = live_table.find_all('div', class_=lambda x: x and 'event__match' in x)
    
    lista_urls = []
    base_url = 'https://www.flashscore.com'
    
    for fila in filas_partidos:
        try:
            # Extraemos el ID del partido desde el atributo 'id' de la fila
            match_id = fila['id'].split('_')[2]
            # Construimos la URL de la página de resumen del partido
            url_partido = f"{base_url}/match/{match_id}/resumen/"
            lista_urls.append(url_partido)
        except Exception:
            continue
            
    print(f"Se recolectaron {len(lista_urls)} URLs de partidos.")
    return lista_urls

# Ejecución de la Fase 1
partidos_urls = obtener_urls_partidos_flashscore(URL_LIGA_CHILENA_2024)

Iniciando Selenium para cargar la tabla dinámica...
Se recolectaron 240 URLs de partidos.


In [36]:
# CÓDIGO A EJECUTAR EN LA CELDA 2 (VERSIÓN FINAL Y COMPLETA)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

# Nota: Debes asegurarte de que URL_LIGA_CHILENA_2024 esté definida en tu Notebook.
# Ejemplo: URL_LIGA_CHILENA_2024 = 'https://www.flashscore.com/football/chile/liga-de-primera-2024/results/'

def obtener_urls_partidos_flashscore(url_liga_general):
    """
    Usa Selenium para cargar la tabla dinámica (incluyendo el clic en "Show more matches") 
    y extrae la URL larga de cada partido.
    """
    print("Iniciando Selenium para cargar la tabla dinámica (Fase 1)...")
    
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options) 
    
    driver.get(url_liga_general)
    driver.implicitly_wait(10)
    
    # 1. HACER CLIC en "Show more matches" para cargar los 240 partidos
    print("Intentando hacer clic en 'Show more matches'...")
    try:
        # Usamos XPATH para encontrar el botón por su texto
        mostrar_mas_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Show more matches')]"))
        )
        # Hacemos clic repetidamente hasta que el botón desaparezca (o falle)
        contador_clics = 0
        while True:
            try:
                driver.execute_script("arguments[0].click();", mostrar_mas_button)
                time.sleep(2) # Pausa para cargar los nuevos datos
                contador_clics += 1
            except Exception:
                break
        print(f"✅ Éxito: Se hicieron {contador_clics} clics en el botón.")
    except Exception:
        print("Botón 'Show more matches' no encontrado o ya cargó todo.")
        pass
    
    # Espera final para asegurar la carga completa de la tabla
    time.sleep(3) 
    
    # 2. Obtener el HTML final
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit() 
    
    live_table = soup.find('div', id='live-table')
    if not live_table: 
        print("Error: La tabla #live-table no se cargó correctamente.")
        return []

    # 3. EXTRACCIÓN Y LIMPIEZA DE LA URL LARGA
    filas_partidos = live_table.find_all('div', class_=lambda x: x and 'event__match' in x)
    lista_urls = []
    
    for fila in filas_partidos:
        try:
            enlace_tag = fila.find('a', class_='eventRowLink', href=True) 

            if enlace_tag and 'href' in enlace_tag.attrs:
                slug_completo = enlace_tag['href']
                
                # Corregimos la URL base
                if slug_completo.startswith('http'):
                    url_partido = slug_completo
                else:
                    url_partido = f"https://www.flashscore.com{slug_completo}"
                
                # Eliminamos el token MID (?mid=...)
                if '?' in url_partido:
                    url_partido = url_partido.split('?')[0]
                
                lista_urls.append(url_partido)
            
        except Exception:
            continue
            
    print(f"Se recolectaron {len(lista_urls)} URLs de partidos (formato largo).")
    return lista_urls

# Ejecución de la Fase 1
partidos_urls = obtener_urls_partidos_flashscore(URL_LIGA_CHILENA_2024)

Iniciando Selenium para cargar la tabla dinámica (Fase 1)...
Intentando hacer clic en 'Show more matches'...
✅ Éxito: Se hicieron 2 clics en el botón.
Se recolectaron 240 URLs de partidos (formato largo).


In [42]:
# CÓDIGO PARA VER LAS PRIMERAS 10 URLS RECOLECTADAS
import pandas as pd

if 'partidos_urls' in locals() and partidos_urls:
    print(f"Total de URLs recolectadas: {len(partidos_urls)}")
    print("--- Primeras 10 URLs (Debe ser formato largo con nombres) ---")
    
    # 1. Mostrar la lista directamente
    for i, url in enumerate(partidos_urls[:11]):
        print(f"{i+1}: {url}")
        
    # 2. Opcional: Crear un DataFrame para un análisis rápido
    df_urls = pd.DataFrame(partidos_urls, columns=['URL_Larga_Flashscore'])
    print("\nEjemplo de la estructura del DataFrame:")
    display(df_urls.head())
    
else:
    print("❌ Error: La lista 'partidos_urls' no existe. Ejecuta la Celda 2 primero.")

Total de URLs recolectadas: 240
--- Primeras 10 URLs (Debe ser formato largo con nombres) ---
1: https://www.flashscore.com/match/football/colo-colo-th4HPIws/copiapo-hxNnWi6H/
2: https://www.flashscore.com/match/football/everton-Kr3LOxgm/u-de-chile-xW771C6U/
3: https://www.flashscore.com/match/football/huachipato-SlhZpIaP/nublense-vVvwgdNn/
4: https://www.flashscore.com/match/football/cobreloa-KhQZzyw6/o-higgins-hYrshGxg/
5: https://www.flashscore.com/match/football/a-italiano-rN3Jxc8g/palestino-U35cjf75/
6: https://www.flashscore.com/match/football/cobresal-YeUwzehC/union-la-calera-M34g8ulJ/
7: https://www.flashscore.com/match/football/deportes-iquique-IsMuMZ5C/u-espanola-p2dQLFin/
8: https://www.flashscore.com/match/football/coquimbo-hWejMZ6h/u-catolica-Qe432hiO/
9: https://www.flashscore.com/match/football/colo-colo-th4HPIws/deportes-iquique-IsMuMZ5C/
10: https://www.flashscore.com/match/football/nublense-vVvwgdNn/u-de-chile-xW771C6U/
11: https://www.flashscore.com/match/football/a-

,URL_Larga_Flashscore
0,https://www.flashscore.com/match/football/colo...
1,https://www.flashscore.com/match/football/ever...
2,https://www.flashscore.com/match/football/huac...
3,https://www.flashscore.com/match/football/cobr...
4,https://www.flashscore.com/match/football/a-it...


In [43]:
# CÓDIGO FINAL CORREGIDO PARA LA CELDA 3 (SINTAXIS Y LÓGICA)

def extraer_estadisticas_detalladas_selenium(url_base):
    """
    Usa Selenium para cargar el contenido dinámico y extraer todas las estadísticas.
    """
    if not url_base.endswith('/'):
        url_base += '/' 
    url_stats = f"{url_base}estadisticas-del-partido/" 
    datos = {'URL': url_stats, 'Local': None, 'Visitante': None, 'Goles_Local': None, 'Goles_Visitante': None}
    
    driver = None # Inicializamos el driver a None para el finally

    # 1. INICIALIZACIÓN DEL DRIVER (Bloque que debe ser lo más limpio posible)
    try:
        print("FLAG 1: Intentando inicializar WebDriver...")
        options = Options()
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("FLAG 2: WebDriver inicializado correctamente.")
        
    except Exception as e:
        print(f"❌ FALLO CRÍTICO 1: Error al iniciar el WebDriver. Causa: {e}")
        return None

    # 2. NAVEGACIÓN Y EXTRACCIÓN DE DATOS (Bloque Principal de Trabajo)
    try:
        driver.get(url_stats)
        print(f"FLAG 3: Navegando a {url_stats}")
        
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.tabContent_match-statistics')))
        print("FLAG 4: Contenedor de estadísticas cargado.")
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # --- A. Extracción de Goles y Nombres ---
        try:
            # Goles y Nombres
            score_wrapper = soup.find('div', class_='detailScore__wrapper')
            score_tags = score_wrapper.find_all('span', recursive=False)
            datos['Goles_Local'] = int(score_tags[0].text)
            datos['Goles_Visitante'] = int(score_tags[2].text)
            
            datos['Local'] = soup.find('div', class_=lambda x: x and 'duelParticipant__home' in x).text.strip()
            datos['Visitante'] = soup.find('div', class_=lambda x: x and 'duelParticipant__away' in x).text.strip()
            print("FLAG 5: Cabecera (Goles/Nombres) extraída con éxito.")
            
        except Exception as e:
            # Capturamos fallos en los selectores de Cabecera.
            print(f"❌ FALLO 5C: Error al extraer la Cabecera. Detalle: {e}")
            # Si los nombres fallan, la fila es inútil, devolvemos None
            if datos['Local'] is None: return None
            # Si solo los goles fallan, continuamos con los datos avanzados
            
        # --- B. Extracción de la Tabla de Estadísticas (Avanzadas) ---
        stats_container = soup.find('div', class_='tabContent_match-statistics')
        if stats_container:
            stat_rows = stats_container.find_all('div', class_=lambda x: x and re.search(r'wcl-row', x))
            
            for row in stat_rows:
                # [Lógica de extracción de valores de estadísticas avanzadas aquí]
                # ... (resto del código de extracción de valores que ya tienes) ...
                pass
            print("FLAG 6: Extracción avanzada completada.")
        
        return datos

    except Exception as e:
        print(f"❌ FALLO 7: Error de Navegación/Timeout. Causa: {e}")
        return None
        
    finally:
        if driver:
            driver.quit()

In [ ]:
# CÓDIGO A EJECUTAR EN LA CELDA 4
# Asegúrate de que las Celdas 1, 2, y 3 se hayan ejecutado previamente.

def crear_dataset_completo(lista_urls):
    """
    Ejecuta el scraper detallado para toda la lista de URLs, 
    consolida los resultados y calcula la variable Target final.
    """
    datos_totales = []
    
    print(f"Iniciando extracción detallada de {len(lista_urls)} partidos...")
    
    for i, url in enumerate(lista_urls):
        # La extracción detallada de las estadísticas ocurre aquí.
        datos = extraer_estadisticas_detalladas_selenium(url) 
        
        if datos:
            datos_totales.append(datos)
        
        # Muestra el progreso cada 20 partidos
        if (i + 1) % 20 == 0:
            print(f"Progreso: {i + 1}/{len(lista_urls)} partidos procesados. Datos recolectados: {len(datos_totales)}")
        
        # Pausa por cortesía (vital para el web scraping)
        time.sleep(2) 

    df_dataset_crudo = pd.DataFrame(datos_totales)
    return df_dataset_crudo

# --- EJECUCIÓN FINAL ---

if 'partidos_urls' in locals() and partidos_urls:
    
    print("\n--- Ejecutando Bucle Completo (¡Puede tardar varios minutos!) ---")
    df_dataset_crudo = crear_dataset_completo(partidos_urls)

    # 1. Creación de la Variable Objetivo (Target)
    def obtener_resultado(row):
        # Usamos los goles que extrajimos de la función (Goles_Local es int, o None si falló)
        if row['Goles_Local'] > row['Goles_Visitante']:
            return 'Local'
        elif row['Goles_Local'] < row['Goles_Visitante']:
            return 'Visitante'
        return 'Empate' # Si son iguales, es empate

    df_dataset_crudo['Target'] = df_dataset_crudo.apply(obtener_resultado, axis=1)

    print("\n" + "=" * 50)
    print(f"✅ Extracción Finalizada. Total de partidos con datos ÚTILES: {len(df_dataset_crudo)}")
    print("El dataset está listo para el preprocesamiento (cálculo de promedios).")
    print("=" * 50)
    
    # 2. Muestra y Guarda el resultado
    display(df_dataset_crudo[['Local', 'Visitante', 'Goles_Local', 'Goles_Visitante', 'ball_possession_local', 'total_shots_local', 'Target']].head())
    
    df_dataset_crudo.to_csv('dataset_flashscore_final.csv', index=False)
    
else:
    print("❌ Error: La lista 'partidos_urls' está vacía. Ejecuta la Celda 2 y verifica la recolección.")